In [848]:
import pandas as pd

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from scipy import stats
import lightgbm as lgb
from scipy.stats import mode
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression

In [849]:
df = pd.read_csv("sources/CreditPrediction.csv")

In [850]:
z_dataset = pd.DataFrame()
df = df.drop(['CLIENTNUM','Unnamed: 19', 'Total_Ct_Chng_Q4_Q1', 'Months_on_book', 'Education_Level', 'Marital_Status', 'Contacts_Count_12_mon', 'Customer_Age', 'Months_Inactive_12_mon'], axis=1)

z_dataset['Total_Trans_Amt'] = np.abs(stats.zscore(df['Total_Trans_Amt']))
outlAmt = list(np.where(np.abs(z_dataset['Total_Trans_Amt']) > 3)[0])
df = df.drop(df.index[outlAmt])



# z_dataset['Total_Trans_Ct'] = np.abs(stats.zscore(df['Total_Trans_Ct']))
# outlct = list(np.where(np.abs(z_dataset['Total_Trans_Ct']) > 3)[0])
# df = df.drop(df.index[outlct])

missing_values = ["NaN", "Unknown", ""]
df.replace(missing_values, np.nan, inplace=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9772 entries, 0 to 10166
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Gender                    9577 non-null   object 
 1   Dependent_count           9772 non-null   int64  
 2   Income_Category           8680 non-null   object 
 3   Card_Category             7922 non-null   object 
 4   Total_Relationship_Count  9753 non-null   float64
 5   Credit_Limit              9772 non-null   float64
 6   Total_Revolving_Bal       9772 non-null   int64  
 7   Total_Amt_Chng_Q4_Q1      9772 non-null   float64
 8   Total_Trans_Amt           9772 non-null   int64  
 9   Total_Trans_Ct            9772 non-null   int64  
 10  Avg_Utilization_Ratio     9772 non-null   float64
dtypes: float64(4), int64(4), object(3)
memory usage: 916.1+ KB


In [851]:
numeric_columns = df.select_dtypes(include=[float, int]).columns

In [852]:
categorical_features = df.select_dtypes(include=['object']).columns

df['Income_Category'] = df['Income_Category'].map({"Less than $40K":0, "$40K - $60K":4, "$60K - $80K":6 ,"$80K - $120K":8, "$120K +":10})
df['Card_Category'] = df['Card_Category'].map({"Blue":0, "Silver":1, "Gold":2, "Platinum":3})
df['Gender'] = df['Gender'].map({"M":0, "F":1})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9772 entries, 0 to 10166
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Gender                    9577 non-null   float64
 1   Dependent_count           9772 non-null   int64  
 2   Income_Category           8680 non-null   float64
 3   Card_Category             7922 non-null   float64
 4   Total_Relationship_Count  9753 non-null   float64
 5   Credit_Limit              9772 non-null   float64
 6   Total_Revolving_Bal       9772 non-null   int64  
 7   Total_Amt_Chng_Q4_Q1      9772 non-null   float64
 8   Total_Trans_Amt           9772 non-null   int64  
 9   Total_Trans_Ct            9772 non-null   int64  
 10  Avg_Utilization_Ratio     9772 non-null   float64
dtypes: float64(7), int64(4)
memory usage: 916.1 KB


In [853]:
# import pandas as pd
# import numpy as np
# from sklearn.cluster import KMeans
# from sklearn.impute import SimpleImputer

# # مرحله 2: مقداردهی اولیه داده‌های مفقود با میانگین
# imputer = SimpleImputer(strategy='mean')
# data_imputed = imputer.fit_transform(df)

# # تبدیل داده‌های پر شده به DataFrame
# df_imputed = pd.DataFrame(data_imputed, columns=df.columns)

# # نمایش داده‌های پر شده با میانگین


# # مرحله 3: اجرای الگوریتم K-means
# kmeans = KMeans(n_clusters=4, random_state=42)
# clusters = kmeans.fit_predict(data_imputed) 

# # اضافه کردن برچسب خوشه به داده‌ها
# df_imputed['cluster'] = clusters



# # مرحله 4: جایگزینی داده‌های مفقود
# for feature in df.columns:
#     if df[feature].isnull().any():
#         for cluster in range(kmeans.n_clusters):
#             # محاسبه میانگین ویژگی برای هر خوشه
#             cluster_mean = df_imputed[df_imputed['cluster'] == cluster][feature].mean()
#             # جایگزینی داده‌های مفقود با میانگین خوشه
#             df_imputed.loc[(df_imputed['cluster'] == cluster) & (df[feature].isnull()), feature] = cluster_mean

# # حذف ستون برچسب خوشه
# df_final = df_imputed.drop(columns=['cluster'])

# # نمایش داده‌های نها

# df = df_final




imputer = KNNImputer(n_neighbors=9)  # You can adjust n_neighbors
df_a = imputer.fit_transform(df)
D = pd.DataFrame(df_a, columns=df.columns)
df = D
df



# from sklearn.experimental import enable_iterative_imputer  # Needed to enable IterativeImputer
# from sklearn.impute import IterativeImputer

# mice_imputer = IterativeImputer(max_iter=20, random_state=42)

# # Fit and transform the data
# df_imputed = mice_imputer.fit_transform(df)

# # Convert the imputed data back to a DataFrame
# df_imputed = pd.DataFrame(df_imputed, columns=df.columns)
# df = df_imputed
# df

,Gender,Dependent_count,Income_Category,Card_Category,Total_Relationship_Count,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio
0,0.0,3.0,6.000000,0.0,5.0,12691.0,777.0,1.335,1144.0,42.0,0.061
1,1.0,5.0,0.000000,0.0,6.0,8256.0,864.0,1.541,1291.0,33.0,0.105
2,0.0,3.0,8.000000,0.0,4.0,3418.0,0.0,2.594,1887.0,20.0,0.000
3,1.0,4.0,0.000000,0.0,3.0,3313.0,2517.0,1.405,1171.0,20.0,0.760
4,0.0,3.0,6.000000,0.0,5.0,4716.0,0.0,2.175,816.0,28.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...
9767,0.0,3.0,8.000000,0.0,5.0,13590.0,1528.0,0.728,2137.0,52.0,0.112
9768,1.0,1.0,0.000000,0.0,2.0,2029.0,1074.0,0.514,4802.0,90.0,0.529
9769,0.0,0.0,6.000000,0.0,3.0,2742.0,2184.0,0.592,3829.0,72.0,0.796
9770,1.0,1.0,2.888889,0.0,1.0,2751.0,1158.0,0.821,4861.0,82.0,0.421


In [870]:
def remove_outliers(df):
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # حذف نقاط دورافتاده
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

df_clean = remove_outliers(df)

# for i, column in enumerate(numeric_columns):
#     plt.figure(figsize=(1, 4))
#     sns.boxplot(y=df[column])
#     plt.title(f'Boxplot for "{column}"')
#     plt.xlabel(column)
#     plt.ylabel('values')
#     plt.show()

# plt.figure(figsize=(18, 24))

# for i, column in enumerate(numeric_columns):
#     plt.subplot(len(numeric_columns), 2, 2*i+1)
#     sns.boxplot(y=df[column])
#     plt.title(f'Before "{column}"')
#     plt.ylabel(column)
    
#     plt.subplot(len(numeric_columns), 2, 2*i+2)
#     sns.boxplot(y=df_clean[column])
#     plt.title(f'After "{column}"')
#     plt.ylabel(column)



df = df_clean
df

,Gender,Dependent_count,Income_Category,Card_Category,Total_Relationship_Count,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio
10,0.0,5.0,10.000000,0.0,5.0,6748.0,1467.0,0.831,1201.0,42.0,0.217
21,1.0,0.0,0.000000,0.0,2.0,1438.3,0.0,1.047,692.0,16.0,0.000
23,1.0,4.0,0.000000,0.0,3.0,2492.0,1560.0,0.573,1126.0,23.0,0.626
25,1.0,3.0,0.000000,0.0,6.0,7768.0,1669.0,0.797,1051.0,22.0,0.215
29,0.0,4.0,4.000000,0.0,6.0,4785.0,1362.0,0.739,1045.0,38.0,0.285
...,...,...,...,...,...,...,...,...,...,...,...
9763,1.0,3.0,4.000000,0.0,4.0,1919.0,774.0,0.965,1505.0,33.0,0.403
9765,1.0,5.0,4.000000,0.0,6.0,4972.0,0.0,0.719,4524.0,72.0,0.000
9768,1.0,1.0,0.000000,0.0,2.0,2029.0,1074.0,0.514,4802.0,90.0,0.529
9769,0.0,0.0,6.000000,0.0,3.0,2742.0,2184.0,0.592,3829.0,72.0,0.796


In [871]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# # مقداردهی اولیه داده‌های مفقود با میانگین
# imputer = SimpleImputer(strategy='mean')
# data_imputed = imputer.fit_transform(df)

# # روش Elbow
# sse = []
# k_range = range(1, 11)
# for k in k_range:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(data_imputed)
#     sse.append(kmeans.inertia_)

# plt.figure(figsize=(10, 6))
# plt.plot(k_range, sse, marker='o')
# plt.xlabel('تعداد کلاسترها')
# plt.ylabel('SSE')
# plt.title('روش Elbow برای تعیین تعداد بهینه کلاسترها')
# plt.show()

In [872]:
# # روش Silhouette
# silhouette_scores = []
# for k in range(2, 11):
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     clusters = kmeans.fit_predict(data_imputed)
#     silhouette_scores.append(silhouette_score(data_imputed, clusters))

# plt.figure(figsize=(10, 6))
# plt.plot(range(2, 11), silhouette_scores, marker='o')
# plt.xlabel('تعداد کلاسترها')
# plt.ylabel('Silhouette Score')
# plt.title('روش Silhouette برای تعیین تعداد بهینه کلاسترها')
# plt.show()

In [873]:
# from sklearn.experimental import enable_iterative_imputer  # Needed to enable IterativeImputer
# from sklearn.impute import IterativeImputer

# mice_imputer = IterativeImputer(max_iter=20, random_state=42)

# # Fit and transform the data
# df_imputed = mice_imputer.fit_transform(df)

# # Convert the imputed data back to a DataFrame
# df_imputed = pd.DataFrame(df_imputed, columns=df.columns)
# df = df_imputed
# df

# imputer = KNNImputer(n_neighbors=5)  # You can adjust n_neighbors
# df_a = imputer.fit_transform(df)
# D = pd.DataFrame(df_a, columns=df.columns)
# df = D
# df

In [874]:
from scipy import stats


# box_cox_transformed, _ = stats.boxcox(df['Customer_Age'])
# df['Customer_Age'] = box_cox_transformed

# box_cox_transformed, _ = stats.boxcox(df['Total_Trans_Amt'])
# df['Total_Trans_Amt'] = box_cox_transformed

# box_cox_transformed, _ = stats.boxcox(df['Total_Trans_Amt'])
# df['Total_Trans_Ct'] = box_cox_transformed

# df

In [875]:
features = df.columns.difference(['Credit_Limit'])  # All columns except 'SalePrice'
target = 'Credit_Limit'

# Convert categorical features to strings

categorical_features = df[features].select_dtypes(include=['object']).columns

for feature in categorical_features:

    df[feature] = df[feature].astype(str)
 
# Split data into features and target

X = df[features]
# StandardScaler
scaler = MinMaxScaler()

# Fit the scaler to the features and transform them
X_scaled = scaler.fit_transform(X)

# محاسبه اطلاعات متقابل بین هر ویژگی و متغیر هدف




y = df[target]
# Split the data into training and testing sets


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
# Find categorical features for CatBoost

categorical_features_indices = np.where(X.dtypes == 'object')[0]


In [876]:
# Check for missing values

missing_values = df.isnull().sum().sort_values(ascending=False)

missing_values = missing_values[missing_values > 0]

print("\nColumns with missing values:\n", missing_values)


Columns with missing values:
 Series([], dtype: int64)


In [877]:
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# تنظیم هایپرپارامترها
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'verbose': -1  # -1 برای کاهش میزان خروجی و جلوگیری از چاپ مکرر
}

num_round = 1000
early_stopping_rounds = 10

bst = lgb.train(
    params,
    train_data,
    num_boost_round=num_round,
    valid_sets=[train_data, valid_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=early_stopping_rounds),
        lgb.log_evaluation(period=100)  # چاپ وضعیت هر 100 دور
    ]
)

y_pred = bst.predict(X_test)
print(mean_squared_error(y_test, y_pred))

Training until validation scores don't improve for 10 rounds
[100]	train's rmse: 1067.24	valid's rmse: 1106.23
[200]	train's rmse: 843.054	valid's rmse: 938.433
[300]	train's rmse: 759.466	valid's rmse: 902.825
Early stopping, best iteration is:
[357]	train's rmse: 728.924	valid's rmse: 897.565
805623.6778900903


In [878]:
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)

test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)
 
# Train the CatBoost model

model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.1,
    eval_metric='RMSE',
    logging_level='Verbose',  # Can be 'Silent' to reduce output
    use_best_model=True, nan_mode='Max'
    
)

# Train the model with evaluation
model.fit(
    train_pool,
    eval_set=test_pool,
    early_stopping_rounds=50  # Stops if validation RMSE doesn't improve for 50 rounds
)

0:	learn: 1641.0588038	test: 1601.3998455	best: 1601.3998455 (0)	total: 1.98ms	remaining: 19.8s
1:	learn: 1568.7325732	test: 1532.4243463	best: 1532.4243463 (1)	total: 4ms	remaining: 20s
2:	learn: 1496.5376547	test: 1463.8666271	best: 1463.8666271 (2)	total: 5.91ms	remaining: 19.7s
3:	learn: 1435.5461787	test: 1406.2182193	best: 1406.2182193 (3)	total: 7.65ms	remaining: 19.1s
4:	learn: 1380.0471938	test: 1354.2348304	best: 1354.2348304 (4)	total: 9.4ms	remaining: 18.8s
5:	learn: 1329.4924289	test: 1305.7712807	best: 1305.7712807 (5)	total: 11.2ms	remaining: 18.6s
6:	learn: 1284.6090618	test: 1264.2840535	best: 1264.2840535 (6)	total: 13ms	remaining: 18.5s
7:	learn: 1243.9356739	test: 1228.2463901	best: 1228.2463901 (7)	total: 14.7ms	remaining: 18.3s
8:	learn: 1204.3875559	test: 1190.4983242	best: 1190.4983242 (8)	total: 16.6ms	remaining: 18.4s
9:	learn: 1171.8128252	test: 1160.4030997	best: 1160.4030997 (9)	total: 18.3ms	remaining: 18.3s
10:	learn: 1137.1615556	test: 1128.6026057	best:

In [879]:
# Make predictions on the test set
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(test_pool)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mse

783314.8789397096

In [880]:
rf_regressor = RandomForestRegressor(max_depth=10)
rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error:', mse)
print('R2 Score:', r2)

Mean Squared Error: 800128.9289572326
R2 Score: 0.7184775596671353
